<a href="https://colab.research.google.com/github/Satwikram/Tensorflow-For-Production/blob/main/Basics/Adding%20Data%20Validation%20to%20Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram K

### Adding Data Validation to Pipelines

### Upgading pip

In [1]:
try:
  import colab
  !pip install -q --upgrade pip
except:
  pass

     |████████████████████████████████| 1.6MB 5.0MB/s 


### Install TFX

In [ ]:
!pip install -q -U --use-deprecated=legacy-resolver tfx==0.27.0

### Basic Imports

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tfx
import os
import shutil

### Basic Dir

In [4]:
SCHEMA_PIPELINE_NAME = 'penguin-tfdv-schema'

In [5]:
PIPELINE_NAME = 'penguin-tfdv'

In [6]:
SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

In [7]:
SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME,
                                    'metadata.db')
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

In [8]:
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [9]:
from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Getting Dataset

In [10]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

('/tmp/tfx-data7vjs_gzz/data.csv', <http.client.HTTPMessage at 0x7fd35d70ee10>)

In [13]:
!head {_data_filepath}

species,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
0,0.2545454545454545,0.6666666666666666,0.15254237288135594,0.2916666666666667
0,0.26909090909090905,0.5119047619047618,0.23728813559322035,0.3055555555555556
0,0.29818181818181805,0.5833333333333334,0.3898305084745763,0.1527777777777778
0,0.16727272727272732,0.7380952380952381,0.3559322033898305,0.20833333333333334
0,0.26181818181818167,0.892857142857143,0.3050847457627119,0.2638888888888889
0,0.24727272727272717,0.5595238095238096,0.15254237288135594,0.2569444444444444
0,0.25818181818181823,0.773809523809524,0.3898305084745763,0.5486111111111112
0,0.32727272727272727,0.5357142857142859,0.1694915254237288,0.1388888888888889
0,0.23636363636363636,0.9642857142857142,0.3220338983050847,0.3055555555555556


### Generate a preliminary schema

In [14]:
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen
from tfx.orchestration import metadata, pipeline

In [15]:
def _create_schema_pipeline(pipeline_name: str, pipeline_root: str, 
                            data_root: str, metadata_path: str) -> pipeline.Pipeline:


  example_gen = CsvExampleGen(input_base = data_root)

  statistics_gen = StatisticsGen(examples = example_gen.outputs['examples'])

  schema_gen = SchemaGen(statistics = statistics_gen.outputs['statistics'], infer_feature_shape=True)

  components = [example_gen, 
                statistics_gen,
                schema_gen]

  return pipeline.Pipeline(
      pipeline_name = pipeline_name,
      pipeline_root = pipeline_root,
      metadata_connection_config = metadata.sqlite_metadata_connection_config(
          metadata_path),
      components=components)

### Run the pipeline

In [16]:
from tfx.orchestration.local import local_dag_runner

In [17]:
local_dag_runner.LocalDagRunner().run(_create_schema_pipeline(

      pipeline_name = SCHEMA_PIPELINE_NAME,

      pipeline_root = SCHEMA_PIPELINE_ROOT,

      data_root = DATA_ROOT, 

      metadata_path = SCHEMA_METADATA_PATH ))  

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Component CsvExampleGen is running.
INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data /tmp/tfx-data7vjs_gzz/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component CsvExampleGen is finished.
INFO:absl:Component StatisticsGen is running.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to pipelines/penguin-tfdv-schema/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to pipelines/penguin-tfdv-schema/StatisticsGen/statistics/2/eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component StatisticsGen is finished.
INFO:absl:Component SchemaGen is running.
INFO:absl:Running driver for SchemaGen
INFO:absl:M

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to pipelines/penguin-tfdv-schema/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component SchemaGen is finished.


### Review outputs of the pipeline

In [18]:
from ml_metadata.proto import metadata_store_pb2
from tfx.orchestration.portable.mlmd import execution_lib
from tfx.types import standard_component_specs


def get_latest_executions(metadata, pipeline_name):
  """Gets the execution objects for the latest run of the pipeline."""
  run_contexts = [
        c for c in metadata.store.get_contexts_by_type('run')
        if c.properties['pipeline_name'].string_value == pipeline_name
    ]
  if not run_contexts:
    return []
  # Pick the latest run context.
  latest_context = max(run_contexts,
                       key=lambda c: c.last_update_time_since_epoch)
  return metadata.store.get_executions_by_context(latest_context.id)

def get_artifacts_for_component_id(metadata, executions, component_id):
  for execution in executions:
    if execution.properties['component_id'].string_value == component_id:
      return execution_lib.get_artifacts_dict(metadata, execution.id,
                                              metadata_store_pb2.Event.OUTPUT)
  raise RuntimeError(f'Execution not found for {component_id}')

from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

from tfx.types import standard_artifacts

In [19]:
metadata_connection_config = metadata.sqlite_metadata_connection_config(
              SCHEMA_METADATA_PATH)

with metadata.Metadata(metadata_connection_config) as metadata_handler:
  # Search component executions from the previous pipeline run.
  executions = get_latest_executions(metadata_handler, SCHEMA_PIPELINE_NAME)

  # Find output artifacts from MLMD.
  stat_gen_output = get_artifacts_for_component_id(metadata_handler, executions,
                                                   'StatisticsGen')
  stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

  schema_gen_output = get_artifacts_for_component_id(metadata_handler,
                                                     executions, 'SchemaGen')
  schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

INFO:absl:MetadataStore with DB connection initialized


In [20]:
visualize_artifacts(stats_artifacts)

In [21]:
visualize_artifacts(schema_artifacts)

,Type,Presence,Valency,Domain
Feature name,,,,
'body_mass_g',FLOAT,required,,-
'culmen_depth_mm',FLOAT,required,,-
'culmen_length_mm',FLOAT,required,,-
'flipper_length_mm',FLOAT,required,,-
'species',INT,required,,-


### Export the schema for future use

In [22]:
_schema_filename = 'schema.pbtxt'
SCHEMA_PATH = 'schema'

os.makedirs(SCHEMA_PATH, exist_ok=True)
_generated_path = os.path.join(schema_artifacts[0].uri, _schema_filename)

# Copy the 'schema.pbtxt' file from the artifact uri to a predefined path.
shutil.copy(_generated_path, SCHEMA_PATH)

'schema/schema.pbtxt'

### Validate input examples and train an ML model

In [23]:
_trainer_module_file = 'penguin_trainer.py'